In [1]:
from H20_Nakata_model import *

In [2]:
import datetime

import numpy as np
import pandas as pd
import pandas.io.data as web
import quandl


def setdata(s, e, t='Adj Close'):
    start = datetime.datetime(s[0], s[1], s[2])
    end = datetime.datetime(e[0], e[1], e[2])
    N225 = web.DataReader('^N225', 'yahoo', start, end)[t]
    DJI = web.DataReader('^DJI', 'yahoo', start, end)[t]
    IXIC = web.DataReader('^IXIC', 'yahoo', start, end)[t]
    jpy = web.DataReader('DEXJPUS', 'fred')
    jpy = jpy.ix[start:end]
    usdjpy = quandl.get("BOE/XUDLJYD", start_date=start, end_date=end).assign(
                    USDJPY_roc = lambda df: df["Value"].pct_change()
                ).rename(columns={
                    "Value" : "USDJPY"
                })
    eurjpy = quandl.get("BOE/XUDLBK63", start_date=start, end_date=end).assign(
                    EURJPY_roc = lambda df: df["Value"].pct_change()
                ).rename(columns={
                    "Value" : "EURJPY"
                })
    gbpjpy = quandl.get("BOE/XUDLJYS", start_date=start, end_date=end).assign(
                    EURJPY_roc = lambda df: df["Value"].pct_change()
                ).rename(columns={
                    "Value" : "GBPJPY"
                })
    usdjpy = usdjpy["USDJPY"]
    eurjpy = eurjpy["EURJPY"]
    gbpjpy = gbpjpy["GBPJPY"]
    
    dataframe = pd.concat([N225, DJI, IXIC, usdjpy, eurjpy, gbpjpy], axis=1).dropna()
    
    datas = np.array(dataframe)
    target = datas.T[0]

    return datas, target

/Users/NIGG/anaconda/lib/python3.5/site-packages/pandas/io/data.py:33: FutureWarning: 
The pandas.io.data module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)


In [3]:
data, target = setdata((2002, 10, 1), (2006, 4, 1))
data

array([[ 11723.629883,  10404.299805,   1984.810059,    107.38    ,
           139.18    ,    202.8193  ],
       [ 11667.540039,  10421.139648,   1991.069946,    108.33    ,
           138.94    ,    202.8154  ],
       [ 11774.30957 ,  10458.459961,   1999.319946,    108.5     ,
           139.29    ,    203.6328  ],
       ..., 
       [ 16938.410156,  11215.700195,   2337.780029,    118.1     ,
           141.72    ,    204.75    ],
       [ 17045.339844,  11150.700195,   2340.820068,    117.22    ,
           142.35    ,    204.631   ],
       [ 17059.660156,  11109.320312,   2339.790039,    117.97    ,
           142.77    ,    204.619   ]])

In [4]:
for i in range(len(data.T)):
    data.T[i] = (data.T[i] - np.average(data.T[i]))/np.max(abs(data.T[i] - np.average(data.T[i])))

target = (target - np.average(target))/np.max(abs(target - np.average(target)))

日経平均とその他のデータとの相関。

In [5]:
print(np.corrcoef(data.T[0], data.T[1])[0, 1])
print(np.corrcoef(data.T[0], data.T[2])[0, 1])
print(np.corrcoef(data.T[0], data.T[3])[0, 1])
print(np.corrcoef(data.T[0], data.T[4])[0, 1])
print(np.corrcoef(data.T[0], data.T[5])[0, 1])

0.815115223996
0.866370655774
0.872475524899
0.766890210222
0.722850448569


In [75]:
print(np.corrcoef(case_A.T[0], case_A.T[1])[0, 1])

0.0435197949283


In [76]:
print(np.corrcoef(case_A.T[0], case_A.T[3])[0, 1])

0.0174362627385


In [6]:
pred = data[-1]
pred

array([ 1.        ,  0.6937186 ,  0.7103626 ,  0.55246872,  0.73558563,
        0.2457783 ])

訓練用のデータとそのラベルの作成

In [7]:
case_A, case_B, case_C, case_D = make_input(data[:-1])
label = make_label(target)

In [8]:
y_A = label[1:]
y_B = label[5:]
y_C = label[5:]
y_D = label[5:]

テスト用のデータとそのラベルの作成

In [9]:
test_data, test_target = setdata((2007, 1, 1), (2007, 12, 1))

In [10]:
for i in range(len(data.T)):
    test_data.T[i] = (test_data.T[i] - np.average(test_data.T[i]))/np.max(abs(test_data.T[i] - np.average(test_data.T[i])))

test_target = (test_target - np.average(test_target))/np.max(abs(test_target - np.average(test_target)))

In [11]:
test_case_A, test_case_B, test_case_C, test_case_D = make_input(test_data[:-1])

In [12]:
test_label = make_label(test_target)
test_label_A = test_label[1:]
test_label_B = test_label[5:]
test_label_C = test_label[5:]
test_label_D = test_label[5:]

In [13]:
P_A = PredictDirection_PriceChanges(case_A, y_A, pred)
P_A.main()

(lnsigma^2, lnC)= (-10, -10) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -9) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -8) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -7) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -6) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -5) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -4) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -3) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -2) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -1) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 0) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 1) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 2) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 3) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 4) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 5) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 6) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 7) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 8) 識別誤り率= 0

(0.04678402279732663, 19.089943925463718, 0.6782608695652174)

In [14]:
P_A.descript(test_case_A, test_label_A)

lnσ^2= -3.06221352757
lnC= 2.94916170036
識別誤り率は 0.32173913043478264
識別成功率は 0.616822429906542
[[ 98.  66.]
 [ 16.  34.]]
グリッドサーチ最高点での識別成功率は 0.616822429906542


0.616822429906542

In [15]:
P_B = PredictDirection_PriceChanges(case_B, y_B, pred)
P_B.main()

(lnsigma^2, lnC)= (-10, -10) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -9) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -8) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 0) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 8) 識別誤り率= 0

(0.40018226975689819, 26636.909413882539, 0.6304347826086957)

In [16]:
P_B.descript(test_case_B, test_label_B)

lnσ^2= -0.91583516127
lnC= 10.1900531049
識別誤り率は 0.3695652173913043
識別成功率は 0.6190476190476191
[[ 85.  53.]
 [ 27.  45.]]
グリッドサーチ最高点での識別成功率は 0.6190476190476191


0.6190476190476191

In [17]:
P_C = PredictDirection_PriceChanges(case_C, y_C, pred)
P_C.main()

(lnsigma^2, lnC)= (-10, -10) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -9) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -8) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 0) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 8) 識別誤り率= 0

(0.16376568502823907, 651.92196613668341, 0.6217391304347826)

In [18]:
P_C.descript(test_case_C, test_label_C)

lnσ^2= -1.80931862262
lnC= 6.47992487092
識別誤り率は 0.37826086956521743
識別成功率は 0.6238095238095238
[[ 86.  53.]
 [ 26.  45.]]
グリッドサーチ最高点での識別成功率は 0.6190476190476191


0.6238095238095238

In [19]:
P_D = PredictDirection_PriceChanges(case_D, y_D, pred)
P_D.main()

(lnsigma^2, lnC)= (-10, -10) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -9) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -8) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 0) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 8) 識別誤り率= 0

(0.064194761368760497, 0.42416304978570535, 0.591304347826087)

In [20]:
P_D.descript(test_case_D, test_label_D)

lnσ^2= -2.74583367023
lnC= -0.85763734629
識別誤り率は 0.408695652173913
識別成功率は 0.5666666666666667
[[ 107.   86.]
 [   5.   12.]]
グリッドサーチ最高点での識別成功率は 0.5666666666666667


0.5666666666666667

In [25]:
random_data = data

for i in range(1, len(data)):
    random_data[i] = [np.random.uniform() for j in range(len(random_data[i]))]

In [26]:
random_data

array([[-0.52204326, -0.34388324, -0.68341739, -0.69587173,  0.20844921,
         0.0775576 ],
       [ 0.1692904 ,  0.40852104,  0.70618881,  0.42342205,  0.32827136,
         0.6108384 ],
       [ 0.81673615,  0.71422132,  0.71016626,  0.35508424,  0.37052796,
         0.19082531],
       ..., 
       [ 0.77796056,  0.93911189,  0.64303985,  0.99996665,  0.01437938,
         0.14670289],
       [ 0.81164588,  0.32966567,  0.26733781,  0.52886898,  0.45854161,
         0.02333662],
       [ 0.97844494,  0.48259254,  0.92543957,  0.96892769,  0.49101563,
         0.96004172]])

In [27]:
random_case_A, random_case_B, random_case_C, random_case_D = make_input(random_data[:-1])

In [28]:
random_case_A

array([[ 0.69133366,  0.75240429,  1.3896062 ,  1.11929378,  0.11982215,
         0.5332808 ],
       [ 0.64744575,  0.30570028,  0.00397745, -0.06833781,  0.0422566 ,
        -0.42001309],
       [-0.30626623, -0.14553549,  0.13977865,  0.08880244,  0.19628368,
         0.54230637],
       ..., 
       [-0.19741703,  0.28499106, -0.47554888,  0.40426118, -0.13827362,
        -0.00839254],
       [ 0.35023723,  0.31431723,  0.53632009,  0.31199851, -0.26133473,
        -0.06872217],
       [ 0.03368531, -0.60944622, -0.37570203, -0.47109767,  0.44416224,
        -0.12336626]])

In [29]:
random_test_data = test_data

for i in range(1, len(test_data)):
    random_test_data[i] = [np.random.uniform() for j in range(len(random_test_data[i]))]

In [30]:
random_test_case_A, random_test_case_B, random_test_case_C, random_test_case_D = make_input(random_test_data[:-1])

In [40]:
random_P_A = PredictDirection_PriceChanges(random_case_A, y_A, pred)
random_P_A.main()

(lnsigma^2, lnC)= (-10, -10) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -9) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -8) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -7) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -6) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -5) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -4) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -3) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -2) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -1) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 0) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 1) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 2) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 3) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 4) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 5) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 6) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 7) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 8) 識別誤り率= 0

(0.06269787114452452, 1.7648524874504465, 0.6260869565217392)

In [41]:
random_P_A.descript(random_test_case_A, test_label_A)

lnσ^2= -2.76942778496
lnC= 0.568067110377
識別誤り率は 0.3739130434782608
識別成功率は 0.5327102803738317
[[ 89.  75.]
 [ 25.  25.]]
グリッドサーチ最高点での識別成功率は 0.5373831775700935


0.5327102803738317

In [32]:
random_P_B = PredictDirection_PriceChanges(random_case_B, y_B, pred)
random_P_B.main()

(lnsigma^2, lnC)= (-10, -10) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -9) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -8) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 0) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 8) 識別誤り率= 0

(0.021511282832268618, 1.4428640974642837, 0.5869565217391305)

In [38]:
random_P_B.descript(random_test_case_B, test_label_B)

lnσ^2= -3.8391776986
lnC= 0.366630094805
識別誤り率は 0.4130434782608695
識別成功率は 0.5333333333333333
[[ 103.   89.]
 [   9.    9.]]
グリッドサーチ最高点での識別成功率は 0.5476190476190477


0.5333333333333333

In [34]:
random_P_C = PredictDirection_PriceChanges(random_case_C, y_C, pred)
random_P_C.main()

(lnsigma^2, lnC)= (-10, -10) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -9) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -8) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 0) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 8) 識別誤り率= 0

(0.020048916817152974, 2.7319061564383764, 0.5826086956521739)

In [36]:
random_P_C.descript(random_test_case_C, test_label_C)

lnσ^2= -3.90958015077
lnC= 1.00499959135
識別誤り率は 0.4173913043478261
識別成功率は 0.5428571428571428
[[ 104.   88.]
 [   8.   10.]]
グリッドサーチ最高点での識別成功率は 0.5428571428571428


0.5428571428571428

In [33]:
random_P_D = PredictDirection_PriceChanges(random_case_D, y_D, pred)
random_P_D.main()

(lnsigma^2, lnC)= (-10, -10) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -9) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -8) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 0) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 8) 識別誤り率= 0

(6.103708360622854e-05, 3.328216039247019e-05, 0.5739130434782609)

In [37]:
random_P_D.descript(random_test_case_D, test_label_D)

lnσ^2= -9.70402895053
lnC= -10.3104890285
識別誤り率は 0.4260869565217391
識別成功率は 0.5333333333333333
[[ 112.   98.]
 [   0.    0.]]
グリッドサーチ最高点での識別成功率は 0.5333333333333333


0.5333333333333333

In [60]:
n225_P_A = PredictDirection_PriceChanges(np.array([case_A.T[0], case_A.T[1]]).T, y_A, np.array([pred[0], pred[1]]))
n225_P_A.main()

(lnsigma^2, lnC)= (-10, -10) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -9) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -8) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -7) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -6) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -5) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -4) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -3) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -2) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, -1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 0) 識別誤り率= 0.4565217391304348
(lnsigma^2, lnC)= (-10, 1) 識別誤り率= 0.5130434782608695
(lnsigma^2, lnC)= (-10, 2) 識別誤り率= 0.491304347826087
(lnsigma^2, lnC)= (-10, 3) 識別誤り率= 0.4869565217391304
(lnsigma^2, lnC)= (-10, 4) 識別誤り率= 0.4869565217391304
(lnsigma^2, lnC)= (-10, 5) 識別誤り率= 0.491304347826087
(lnsigma^2, lnC)= (-10, 6) 識別誤り率= 0.4869565217391304
(lnsigma^2, lnC)= (-10, 7) 識別誤り率= 0.4869565217391304
(lnsigma^2, lnC)= (-10, 8) 識別誤り率= 0.4

(0.045662683679675034, 6.349921845828538, 0.6434782608695652)

In [64]:
n225_P_A.descript(np.array([test_case_A.T[0], test_case_A.T[1]]).T, test_label_A)

lnσ^2= -3.08647386432
lnC= 1.84844250509
識別誤り率は 0.35652173913043483
識別成功率は 0.602803738317757
[[ 108.   79.]
 [   6.   21.]]
グリッドサーチ最高点での識別成功率は 0.602803738317757


0.602803738317757

In [65]:
n225_DJ_P_B = PredictDirection_PriceChanges(np.array([case_B.T[0], case_B.T[1]]).T, y_B, np.array([pred[0], pred[1]]))
n225_DJ_P_B.main()

(lnsigma^2, lnC)= (-10, -10) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -9) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -8) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -1) 識別誤り率= 0.4173913043478261
(lnsigma^2, lnC)= (-10, 0) 識別誤り率= 0.4304347826086956
(lnsigma^2, lnC)= (-10, 1) 識別誤り率= 0.4521739130434783
(lnsigma^2, lnC)= (-10, 2) 識別誤り率= 0.4695652173913043
(lnsigma^2, lnC)= (-10, 3) 識別誤り率= 0.4652173913043478
(lnsigma^2, lnC)= (-10, 4) 識別誤り率= 0.4652173913043478
(lnsigma^2, lnC)= (-10, 5) 識別誤り率= 0.4826086956521739
(lnsigma^2, lnC)= (-10, 6) 識別誤り率= 0.4869565217391304
(lnsigma^2, lnC)= (-10, 7) 識別誤り率= 0.4695652173913043
(lnsigma^2, lnC)= (-10, 8) 識別誤り率= 0

(0.18759550879069548, 1.2682200611970336, 0.5956521739130435)

In [66]:
n225_DJ_P_B.descript(np.array([test_case_B.T[0], test_case_B.T[1]]).T, test_label_B)

lnσ^2= -1.67346718304
lnC= 0.237614390797
識別誤り率は 0.4043478260869565
識別成功率は 0.580952380952381
[[ 110.   86.]
 [   2.   12.]]
グリッドサーチ最高点での識別成功率は 0.580952380952381


0.580952380952381

In [67]:
n225_DJ_P_C = PredictDirection_PriceChanges(np.array([case_C.T[0], case_C.T[1]]).T, y_C, np.array([pred[0], pred[1]]))
n225_DJ_P_C.main()

(lnsigma^2, lnC)= (-10, -10) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -9) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -8) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -1) 識別誤り率= 0.4217391304347826
(lnsigma^2, lnC)= (-10, 0) 識別誤り率= 0.4565217391304348
(lnsigma^2, lnC)= (-10, 1) 識別誤り率= 0.4739130434782609
(lnsigma^2, lnC)= (-10, 2) 識別誤り率= 0.4869565217391304
(lnsigma^2, lnC)= (-10, 3) 識別誤り率= 0.491304347826087
(lnsigma^2, lnC)= (-10, 4) 識別誤り率= 0.4739130434782609
(lnsigma^2, lnC)= (-10, 5) 識別誤り率= 0.4739130434782609
(lnsigma^2, lnC)= (-10, 6) 識別誤り率= 0.4869565217391304
(lnsigma^2, lnC)= (-10, 7) 識別誤り率= 0.4869565217391304
(lnsigma^2, lnC)= (-10, 8) 識別誤り率= 0.

(0.90909202998420113, 360.84579375665305, 0.6)

In [68]:
n225_DJ_P_C.descript(np.array([test_case_C.T[0], test_case_C.T[1]]).T, test_label_C)

lnσ^2= -0.0953089468225
lnC= 5.88845070296
識別誤り率は 0.4
識別成功率は 0.580952380952381
[[ 107.   83.]
 [   5.   15.]]
グリッドサーチ最高点での識別成功率は 0.5904761904761905


0.580952380952381

In [69]:
n225_DJ_P_D = PredictDirection_PriceChanges(np.array([case_D.T[0], case_D.T[1]]).T, y_D, np.array([pred[0], pred[1]]))
n225_DJ_P_D.main()

(lnsigma^2, lnC)= (-10, -10) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -9) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -8) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -7) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -6) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -5) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -4) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -3) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -2) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, -1) 識別誤り率= 0.4260869565217391
(lnsigma^2, lnC)= (-10, 0) 識別誤り率= 0.4956521739130435
(lnsigma^2, lnC)= (-10, 1) 識別誤り率= 0.5347826086956522
(lnsigma^2, lnC)= (-10, 2) 識別誤り率= 0.5260869565217392
(lnsigma^2, lnC)= (-10, 3) 識別誤り率= 0.5304347826086957
(lnsigma^2, lnC)= (-10, 4) 識別誤り率= 0.5391304347826087
(lnsigma^2, lnC)= (-10, 5) 識別誤り率= 0.5347826086956522
(lnsigma^2, lnC)= (-10, 6) 識別誤り率= 0.5391304347826087
(lnsigma^2, lnC)= (-10, 7) 識別誤り率= 0.5347826086956522
(lnsigma^2, lnC)= (-10, 8) 識別誤り率= 0

(0.11170178867897641, 7.8360972123891184, 0.6)

In [70]:
n225_DJ_P_D.descript(np.array([test_case_D.T[0], test_case_D.T[1]]).T, test_label_D)

lnσ^2= -2.19192255979
lnC= 2.05874090587
識別誤り率は 0.4
識別成功率は 0.5619047619047619
[[ 111.   91.]
 [   1.    7.]]
グリッドサーチ最高点での識別成功率は 0.5714285714285714


0.5619047619047619